In [190]:
import numpy as np
from sklearn.base import BaseEstimator, clone

class RBT(BaseEstimator):
    
 http://graphics.stanford.edu/~smr/ICP/comparison/eggert_comparison_mva97.pdf    def __init__(self, R = None, T = None):
        self.R = R
        self.T = T

    
    def fit(self, left, right):
        
        mu_left, mu_right = left.mean(0), right.mean(0)
        
        left_ = left - mu_left
        right_ = right - mu_right
        
        corr_H = left_.T @ right_
        U, S, VT = np.linalg.svd(corr_H)
        
        R = VT.T @ U.T
        T = mu_right - R @ mu_left
        
        self.R = R
        self.T = T
        
        return self

    def score(self,  left, right):
        R, T = self.R, self.T        
        projected = (R @ (left).T).T + T
        MSE = np.linalg.norm(right - projected, axis=1).mean()
        return MSE
    
    def predict(self, left):
        R, T = self.R, self.T   
        projected = (R @ (left).T).T + T
        return projected
        

# TEST
from scipy.spatial.transform import Rotation as Rot
test_points = np.random.normal(2, 2, (4, 3))       
R = Rot.from_euler('zyx', [90, 45, 30], degrees=True).as_matrix()
T = np.array([15, 10, 5])        
test_points_trans = (R @ (test_points).T).T + T
rst = RBT()

rst.fit(test_points, test_points_trans)
R, rst.R, T, rst.T, rst.score(test_points, test_points_trans)

(array([[ 1.11022302e-16, -7.07106781e-01,  7.07106781e-01],
        [ 8.66025404e-01, -3.53553391e-01, -3.53553391e-01],
        [ 5.00000000e-01,  6.12372436e-01,  6.12372436e-01]]),
 array([[-1.72508955e-16, -7.07106781e-01,  7.07106781e-01],
        [ 8.66025404e-01, -3.53553391e-01, -3.53553391e-01],
        [ 5.00000000e-01,  6.12372436e-01,  6.12372436e-01]]),
 array([15, 10,  5]),
 array([15., 10.,  5.]),
 2.3117070079741243e-15)

In [191]:
from sklearn.linear_model import RANSACRegressor

inlier_frac = 0.1

outlier = np.random.normal(2, 2, (int(4 // inlier_frac), 3))     
outlier_trans = (R @ (outlier).T).T + T    
np.random.shuffle(outlier_trans)

data_x = np.vstack([test_points, outlier]) 
data_y = np.vstack([test_points_trans, outlier_trans])                   

reg = RANSACRegressor(random_state=0, 
                      base_estimator=RBT(), 
                      min_samples=4, 
                      residual_threshold=0.1, 
                      max_trials=10000, 
                      stop_n_inliers=10)

reg.fit(data_x, data_y)
reg.score(data_x, data_y), reg.estimator_

(3.340062884658763,
 RBT(R=array([[-2.07584781e-17, -7.07106781e-01,  7.07106781e-01],
        [ 8.66025404e-01, -3.53553391e-01, -3.53553391e-01],
        [ 5.00000000e-01,  6.12372436e-01,  6.12372436e-01]]),
     T=array([15., 10.,  5.])))